## Denoise

MRI scans often show “salt-and-pepper” noise—voxels that differ slightly from their neighbors—making surrounding air look hazy and affecting volume rendering. Two common ways to address this are Gaussian blur denoising and removing dark pixels via Otsu thresholding; they can be applied separately or combined. The interactive controls in this Jupyter script demonstrate these techniques and let you explore their effects. The demo is similar to the [NiiVue denoise web page](https://niivue.com/demos/features/denoise.html).


In [ ]:
import ipywidgets as widgets
import numpy as np
from scipy.ndimage import grey_dilation, median_filter

from ipyniivue import NiiVue
from ipyniivue.constants import DragMode, ShowRender
from ipyniivue.utils import find_otsu

nv = NiiVue(
    back_color=(0.9, 0.9, 1, 1),
    show_3d_crosshair=True,
)

nv.load_volumes([{"path": "../images/otsu.nii.gz",}])

imgRaw = None


@nv.on_image_loaded
def on_image_loaded(volume):
    """Handle image loaded."""
    global imgRaw
    imgRaw = volume.img.copy()
    print("Image loaded. Original image data stored.")


@nv.on_location_change
def handle_intensity_change(location):
    """Handle location change."""
    with intensity_output:
        intensity_output.clear_output()
        print(location["string"])


nv.opts.multiplanar_show_render = ShowRender.ALWAYS
nv.opts.yoke_3d_to_2d_zoom = True
nv.set_interpolation(True)
nv.set_clip_plane(0.2, 0, 120)

## Create UI elements


otsu_select = widgets.Dropdown(
    options=[
        ("Very Heavy 3:4", 1),
        ("Heavy 2:3", 2),
        ("Medium 1:2", 3),
        ("Light 1:3", 4),
        ("Very Light 1:4", 5),
        ("None", 6),
    ],
    value=6,
    description="Dehaze:",
)

denoise_check = widgets.Checkbox(value=False, description="Denoise")
dilate_check = widgets.Checkbox(value=True, description="Dilate dark")
dilate_check.layout.display = "none"

save_button = widgets.Button(description="Save")
dark_check = widgets.Checkbox(value=True, description="Clip Dark")

intensity_output = widgets.Output()

## Functions to handle UI events

def on_dark_check_change(change):
    """Handle dark check change."""
    nv.opts.is_alpha_clip_dark = change["new"]


on_dark_check_change({"new": dark_check.value})


def on_drag_mode_change(change):
    """Handle drag mode change."""
    if change.new == "none":
        nv.opts.drag_mode = DragMode.NONE
    elif change.new == "contrast":
        nv.opts.drag_mode = DragMode.CONTRAST
    elif change.new == "measurement":
        nv.opts.drag_mode = DragMode.MEASUREMENT
    elif change.new == "pan":
        nv.opts.drag_mode = DragMode.PAN


drag_mode_dropdown = widgets.Dropdown(
    options=[
        ("Drag pan/zoom", "pan"),
        ("Drag contrast", "contrast"),
        ("Drag measurement", "measurement"),
        ("Drag none", "none"),
    ],
    value="pan",
    description="Drag Mode:",
)
drag_mode_dropdown.observe(on_drag_mode_change, names="value")

def on_save_clicked(b):
    """Save image."""
    nv.volumes[0].save_to_disk("denoised.nii")

## Processing functions

def denoise_image(img_raw, nx, ny, nz):
    """Denoise image."""
    img_raw = img_raw.reshape(nz, ny, nx)
    img_smoothed = median_filter(img_raw, size=3)
    return img_smoothed.flatten()


def dilate_image(img_raw, nx, ny, nz):
    """Dilate image."""
    img_raw = img_raw.reshape(nz, ny, nx)
    img_dilated = grey_dilation(img_raw, size=(3, 3, 3))
    return img_dilated.flatten()


def process_volume(change=None):
    """Process volume."""
    global imgRaw
    if imgRaw is None:
        print("Image not loaded yet.")
        return

    level = otsu_select.value
    # dilate check visibility contingent on dehazing
    if level == 6:
        dilate_check.layout.display = "none"
    else:
        dilate_check.layout.display = ""

    # Reload original image
    img_raw = imgRaw.copy()
    nx = int(nv.volumes[0].dims[1])
    ny = int(nv.volumes[0].dims[2])
    nz = int(nv.volumes[0].dims[3])

    if level in [5, 1]:
        otsu = 4
    elif level in [4, 2]:
        otsu = 3
    else:
        otsu = 2

    thresholds = find_otsu(nv.volumes[0], mlevel=otsu)
    if len(thresholds) < 3:
        print("Threshold calculation failed.")
        return

    threshold = thresholds[0]
    if level == 1:
        threshold = thresholds[2]
    elif level == 2:
        threshold = thresholds[1]

    mn = np.min(img_raw)
    if level > 5:
        threshold = mn

    if denoise_check.value:
        img_raw = denoise_image(img_raw, nx, ny, nz)

    if dilate_check.value and level < 6:
        imgMx = dilate_image(img_raw, nx, ny, nz)
    else:
        imgMx = img_raw.copy()

    # Apply threshold
    mask = imgMx < threshold
    img_processed = img_raw.copy()
    img_processed[mask] = mn

    img_processed = img_processed.astype(imgRaw.dtype)

    nv.volumes[0].img = img_processed

## Attach Handlers
save_button.on_click(on_save_clicked)
dark_check.observe(on_dark_check_change, names="value")
otsu_select.observe(process_volume, names="value")
dilate_check.observe(process_volume, names="value")
denoise_check.observe(process_volume, names="value")

## Display all

ui_header = widgets.VBox(
    [
        widgets.HBox([denoise_check, otsu_select, dilate_check])
    ]
)

ui_footer = widgets.VBox(
    [
        widgets.HBox([save_button, dark_check, drag_mode_dropdown])
    ]
)

display(ui_header)
display(nv)
display(ui_footer)
display(intensity_output)